This notebook is an old version of 
Flag for deletion.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from __future__ import division
import os
import six
import sys
sys.path.append('..'); import pathcustomize, about
import pathcustomize, about
about.about()

import pickle

import math
import pandas as pd
from scipy import stats
import scipy.interpolate as interpolate
import scipy.signal as ss


import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec
from mpltools import style
from mpltools import layout
style.use('ggplot')

import waldo
from waldo import wio
from waldo import viz
import waldo.metrics.report_card as report_card
import waldo.metrics.step_simulation as ssim
import waldo.viz.eye_plots as ep

#from waldo.metrics.report_card import compound_bl_filter

In [ ]:
### small plates
# N = 15
ex_id = '20141017_113435'
ex_id = '20141017_113439'
ex_id = '20141017_123722'
ex_id = '20141017_123725'
# N = 25
ex_id = '20141017_134720'
ex_id = '20141017_134724'
#ex_id = '20141017_150959'
#ex_id = '20141017_151002'

#### big plates
#ex_id = '20130614_120518'
#ex_id = '20130318_131111'
#ex_id = '20130702_135704' # testset
#ex_id = '20130702_135652'
#ex_id = '20130414_140704'
#ex_id = '20130410_165326'


In [ ]:
experiment = wio.Experiment(experiment_id=ex_id)
prep_data = experiment.prepdata
graph = experiment.graph.copy()

In [ ]:
graph2, rc = report_card.iterative_solver(experiment, graph.copy())

In [ ]:
step_min_move = 1

raw_moving_nodes =  [int(i) for i in graph.compound_bl_filter(experiment, threshold=step_min_move)]
raw_steps, raw_durations = report_card.calculate_duration_data_from_graph(experiment, graph, raw_moving_nodes)

final_moving_nodes =  [int(i) for i in graph2.compound_bl_filter(experiment, threshold=step_min_move)]
final_steps, final_durations = report_card.calculate_duration_data_from_graph(experiment, graph2, final_moving_nodes)

### Final Eye Plot

In [ ]:
sim_steps = ssim.run_ideal_step_simulation(experiment, verbose=True)
sim_steps2 = ssim.naive_simulation(experiment, verbose=True)
sim_durations = list(sim_steps2['lifespan'])

In [ ]:
fig, ax = plt.subplots()

short_lim = 5
step_kwargs = {'short_lim':short_lim, 'front_back_margin':10}

ymax = len(final_steps[final_steps['lifespan'] > short_lim])

legend_props = ep.eye_plot(ax, final_steps, 
                           color=5, label='final',
                            ymax=ymax, **step_kwargs)


ax.set_ylabel('track number')
ax.set_xlabel('existence (min)')

ep.add_simulation_lines(ax, raw_steps, ymax=ymax, labels=['raw', ''], color='red')
ep.add_simulation_lines(ax, sim_steps2, ymax=ymax, labels=['simulated', ''], color='black')

ax.legend(**legend_props)
plt.savefig('final-eye-{eid}.png'.format(eid=ex_id))
plt.show()

### Fnial Duration Histogram

In [ ]:
fig, ax = plt.subplots()

bins = np.arange(10, 61, 5) #np.linspace(0, np.percentile(s, 95), n_bins)
#raw, be = np.histogram(raw_durations, bins=bins)
#final, be = np.histogram(final_durations, bins=bins)
sim, be = np.histogram(sim_durations, bins=bins)

print sim
print be
print len(sim)
sim = list(sim)
sim = [sim[0]] + sim 
ax.hist(raw_durations, bins=bins, alpha=0.5, label='raw')
ax.hist(final_durations, bins=bins, alpha=0.5, label='final')
#ax.hist(sim_durations, bins=bins, alpha=0.5)
ax.plot([0,0], [0,0]) #, label='final', drawstyle='steps')
print len(sim), len(bins)
ax.plot(bins, sim, drawstyle='steps', label='simulation')
ax.set_xlabel('duration of tracks (min)')
ax.set_ylabel('track counts')
ax.set_xlim([10, 60])
ax.legend()
plt.savefig('final-hist-{eid}.png'.format(eid=ex_id))
plt.show()

In [ ]:
fig, ax = plt.subplots()

bins = np.arange(30, 61, 5) #np.linspace(0, np.percentile(s, 95), n_bins)
#raw, be = np.histogram(raw_durations, bins=bins)
#final, be = np.histogram(final_durations, bins=bins)
sim, be = np.histogram(sim_durations, bins=bins)

print sim
print be
print len(sim)
sim = list(sim)
sim = [sim[0]] + sim 
#print len(raw), len(bins[:-1])
#print raw
#ax.plot(be[:-1], raw, label='raw', drawstyle='steps')
#ax.plot(be[:-1], final, label='final', drawstyle='steps')
ax.hist(raw_durations, bins=bins, alpha=0.5)
ax.hist(final_durations, bins=bins, alpha=0.5)
#ax.hist(sim_durations, bins=bins, alpha=0.5)
ax.plot([0,0], [0,0]) #, label='final', drawstyle='steps')
print len(sim), len(bins)
ax.plot(bins, sim, drawstyle='steps')
ax.set_xlabel('duration of tracks (min)')
ax.set_ylabel('track counts')
ax.set_xlim([30, 60])
ax.legend()
plt.show()